# Working with Texts

## About this Tutorial
This document is a re-writing of [DARIAH-DE's Working With Texts tutorial](https://de.dariah.eu/tatom/working_with_text.html). Basically, it is an attempt explain things for which I think beginners will need more detail and change the emphasis in places. Readers will find it valuable to compare the text and examples here with the original version.

## Some Concepts and Terminology
A text is a string of characters that can be divided up into tokens. A *tokens* are strings of contiguous characters divided by some kind of boundary, frequently a space or punctuation mark. In English this boundary generally divides the token into words. It is also possible to use string length to divide a text into tokens. For Classical Chinese, for instance, it might be preferable to make every character a separate token. Once a text is divided into tokens, the tokens can be counted. Each countable token is called a *term*.

From a processing point view, texts are made up of tokens and terms. A *token* is an identifiable unit in the text. Anything from individual characters, to words, to phrases can be a token. Tokens are sometimes called an *n-grams*, because they can be a strings of any length. Tokens consisting of single words or characters are unigrams, sequences of two words or characters are bi-grams. For literary text analysis word unigrams are typically treated as tokens. If we list the number of times each token is found in the text, we will have a list of words, each of which is different from all the others. These words are called *terms*. For instance, the phrase "the cat in the hat" has 5 tokens but 4 terms. We can express this nicely in Python:


In [5]:
# The token list
tokens = ['the', 'cat', 'in', 'the', 'hat']
# Print the length (the number of items) in the tokens list
print(len(tokens))

5


In [6]:
# The term counts
terms = {"cat": 1, "hat": 1, "in": 1, "the": 2}
# Print the length (the number of items) in the terms dict
print(len(terms))

4


By using a Python list to represent the tokens, we capture the fact that a text is an ordered list of tokens. A Python dictionary doesn't preserve the original order of items, but for terms, the order doesn't matter (it's alphabetised in the example above). When produce a list of terms and their counts, we essentially turn the text into a "bag of words", in which their order of occurrence is lost.

It is important to think about the implications of studying bags of words. In the process, relationship *between* words (in terms of grammar and/or proximity) will be lost. One way of addressing this problem is to treat tokens as n-grams. An *n-gram* is a token of variable length. If we use single characters or words as tokens, we are using a type of n-gram called a "unigram". But we could also make our tokens sequences of two characters or two words. These are called "bi-grams". We could also make the token boundary every third character or word, giving us "tri-grams". N-grams can be even bigger, in which case the notation tends to shift towards numbers: 4-grams, 5-grams, etc. Although n-grams can be used to address some of the problems we might identify with using bags of unordered words, a great deal of literary text analysis is doen using word unigrams. This can be very effective, as we'll see from the examples below.

A list of term frequencies can also be called a *vector*, borrowing a term from mathematics. Treating texts as vectors of term frequencies enables us to manipulate them using a range of Python functions designed for studying and manipulating vectors. We'll be using two important Python packages for this purpose, `numpy` and `scikit-learn`.

Another important concept is the *document-term matrix* (or DTM). *Document* is a generic term for a textual unit, such as a novel, newspaper article, or poem, which we wish to study. It can also mean a segment of some larger unit. For instance, we might divide a novel into ten equal parts, each of which would be a separate document. We can organise our documents into rows, with each term in a separate column, as we would in a spreadsheet. This is the document-term matrix. Part of a DTM might look like this:

|       | term1 | term2 | term3 |
|-------|------:|------:|------:|
| doc1  | 43    |  34   |   56  |
| doc2  |  2    |  45   |   90  |
| doc3  | 23    |  45   |   78  |

The DTM allows us to compare term frequencies in the documents we are studying.

## Getting Some Data
The DARIAH-DE tutorial provides a data set which can be downloaded by clicking the [datasets.zip](https://de.dariah.eu/tatom/_downloads/datasets.zip) link. Before proceeding, extract the zip file to a place you can remember. We will be using the small collection of novels by Jane Austen and Charlotte Brontë. Open the `data` folder and locate the `austen-brontë` folder. Change the `ë` to `e`. Check inside the folder--there may be further examples. Make sure you know the command-line path to the data folder. For instance, if you are on Windows, it might be `C:\Users\YOURNAME\Desktop\`. For you in a CSUN lab, replace `YOURNAME` with your CSUN ID.

The reason we are removing the `ë` is that it is a [Unicode](https://en.wikipedia.org/wiki/Unicode) character, which may be handled differently by different version of Python. If we change it to `e`, we don't have to worry about this complication.


## Preparing to Analyze Document Vectors
The Python `numpy` package has a special data type that allows us to manipulate a DTM, so we'll begin by importing it (and assign it to the conventional alias alias `np`). We'll also import the `CountVectorizer` functions from `scikit-learn's` text feature extraction suite.

In [2]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

We will use `CountVectorizer` both to load the texts and to construct a DTM from them. We'll start by constructing a list of filenames:

In [3]:
filenames = [
    'C:/Users/Scott/Desktop/datasets/data/austen-bronte/Austen_Emma.txt',
    'C:/Users/Scott/Desktop/datasets/data/austen-bronte/Austen_Pride.txt',
    'C:/Users/Scott/Desktop/datasets/data/austen-bronte/Austen_Sense.txt',
    'C:/Users/Scott/Desktop/datasets/data/austen-bronte/CBronte_Jane.txt',
    'C:/Users/Scott/Desktop/datasets/data/austen-bronte/CBronte_Professor.txt',
    'C:/Users/Scott/Desktop/datasets/data/austen-bronte/CBronte_Villette.txt'
]

Each item in the list is the path to the text file within the data folder you downloaded. You must supply the full path on your disk. For instance, if you are using Windows, it might be `C:\Users\YOURNAME\Desktop\`. Go ahead and add this to each of the file paths. If you are using Windows, you can change the back slashes to forward slashes. iPython Notebook will still find your files. Do this before continuing. And, remember, each of the items in this list is a string, so it must be fully enclosed in apostrophes quotation marks.

Now we'll create a `CountVectorizer` object and tell it that the input will be a file name. We'll assign this to the variable `vectorizer` for short. Next we'll feed each of the files in the `filenames` list into the vectorizer and use the `fit_transform()` function to convert it to a sparse matrix (one in which the count for most terms is 0). We'll assign to a variable called `dtm`. Finally, we'll use the `get_feature_names()` function to get a list of the terms.

In [4]:
vectorizer = CountVectorizer(input='filename')
dtm = vectorizer.fit_transform(filenames)
terms = vectorizer.get_feature_names()

dtm = dtm.toarray()  # convert the sparse matrix to a numpy array
terms = np.array(terms) # Do the same with the terms list

The last two lines convert our dtm and terms list to numpy arrays, which make it possible to query them for the frequencies of individual terms. Let's try this. Assume that we want to find the frequency of the word "house" in *Emma*. This is the 0th text in our filenames list (you can confirm this with `print(filenames[0]))`). We next construct an index for the word "house".

In [4]:
house_idx = list(terms).index('house')
house_count = dtm[0, house_idx]
print(house_count)

# An alternative approach
house_count = dtm[0, terms == 'house'][0]
print(house_count)

95
95


The first method above fetches a list of the terms in our DTM and finds the key, or index, of "house" in the list. With that information, we can supply it with the index of *Emma* (0) to find the count for "house" in that text. The second code block does the same thing with `numpy's` method of indexing. This returns a list containing first the answer we want and then some other information about the shape of the array. By adding the `[0]` index to the end, we can get Python to print only the first item. You can use which over of these two methods you find most intuitive.

Just so we have a sense of what we have just created, here is a section of the document-term matrix for a handful of selected words:

|                     |and  |emma |home |house|of  |the |
|:--------------------|----:|----:|----:|----:|---:|---:|
|Austen_Emma.txt      |4896 |865  |130  |95   |4291|5201|
|Austen_Pride.txt     |3584 |0    |66   |107  |3609|4330|
|Austen_Sense.txt     |3491 |0    |69   |161  |3572|4105|
|CBronte_Jane.txt     |6626 |0    |80   |182  |4364|7846|
|CBronte_Professor.txt|2936 |0    |37   |93   |2663|3836|
|CBronte_Villette.txt |6374 |0    |121  |129  |4845|8363|

## Comparing Texts
Since each row of the document-term matrix is a sequence of a novel's word frequencies, it is possible to put mathematical notions of similarity (or distance) between sequences of numbers in service of calculating the similarity (or distance) between any two novels. One frequently used measure of distance between vectors (a measure easily converted into a measure of similarity) is Euclidean distance. Imagine two document vectors as straight lines starting from the bottom left of a graph. If you draw a straight line connecting the two vectors and measure it, you have the Euclidean distance. Since `scikit-learn` has some built-in functions for calculating Euclidean distances, we'll go ahead and import it.

In [5]:
from sklearn.metrics.pairwise import euclidean_distances
dist = euclidean_distances(dtm)
np.round(dist, 1)

array([[    0. ,  3856.3,  4182.8,  5119.7,  7113.3,  5280.2],
       [ 3856.3,     0. ,  1922.6,  6313.1,  4126.2,  6381.2],
       [ 4182.8,  1922.6,     0. ,  6657.4,  4045.3,  6650.3],
       [ 5119.7,  6313.1,  6657.4,     0. ,  8363.8,  2591.5],
       [ 7113.3,  4126.2,  4045.3,  8363.8,     0. ,  8484.1],
       [ 5280.2,  6381.2,  6650.3,  2591.5,  8484.1,     0. ]])

Here we are creating a new distance matrix (also called a "dissimilarity matrix") from out DTM. It tells us the distances between each of our document vectors. The calculated results are likely to have lots of decimal places, so the last line just uses a `numpy` function to round the numbers to one decimal places. You can see this in the result.

In our list of novels, *Pride and Prejudice* is index 1 and *Jane Eyre* is index 3. We can compare their distances by querying our distance matrix:

In [10]:
# Distance between Pride and Prejudice and Jane Eyre
dist[1, 3]

6313.0833987838305

Try comparing some of the other novels. What can the Euclidean distances tell you about the novels?

Another common measure used for comparing texts is Cosine similarity, which works by measuring the angle between the two document vectors. If you want to try this, you can get the code from lines 24-28 of the [original tutorial](https://de.dariah.eu/tatom/working_with_text.html). We will use Euclidean distance for the visualisation and clustering techniques we learn below.

## Visualising Document Distances
What if we want to compare all the texts at once? If you compared the distances of all the novels, you might have done this, but the implications of the numbers can sometimes be hard to interpret. This is where visualisatio can be useful. If we interpret these distances as x and y coordinates on a graph, we can plot how far apart texts are. A general approach to visualizing distances is to assign a point in a plane to each text, making sure that the distance between points is proportional to the pairwise distances we calculated. This kind of visualization is common enough that it has a name, "multidimensional scaling (MDS) and family of functions in `scikit-learn`.

Before we start, we need to take care of a problem will encounter in creating a graph. Right now, our texts are take the form of long path names. We really want something shorter for use as labels in the graph. So we'll create a list of short labels:

In [6]:
labels = [
    'Austen_Emma',
    'Austen_Pride',
    'Austen_Sense',
    'CBronte_Jane',
    'CBronte_Professor',
    'CBronte_Villette'
]

from sklearn.manifold import MDS
import matplotlib.pyplot as plt

We've also inmported `scikit-learn's` MDS functions and the `maplotlib` plotting library (conventionally) referenced as `plt`. The following two lines will create a matrix of positions. A few explanations are given in comments, but don't worry too much about the details.

In [7]:
# two components as we're plotting points in a two-dimensional plane
# "precomputed" because we provide a distance matrix
# we will also specify `random_state` so the plot is reproducible.
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
pos = mds.fit_transform(dist)  # shape (n_components, n_samples)
xs, ys = pos[:, 0], pos[:, 1]

Now we're ready to plot our graph (which will open in another window). The code below creates a scatter plot. By default, the dots on the graph will be sky blue, but the colour is set to orange if the label contains the string "Austen".

In [ ]:
# color-blind-friendly palette
for x, y, label in zip(xs, ys, labels):
    color = 'orange' if "Austen" in label else 'skyblue'
    plt.scatter(x, y, c=color)
    plt.text(x, y, label)
plt.show()

Check out the [original tutorial](https://de.dariah.eu/tatom/working_with_text.html) for a way to plot the graph in 3D (but note that we have changed some of the variable names here, so you'll have to adjust the code.)

## Cluster Analysis
Clustering texts into discrete groups of similar texts is often a useful exploratory step. For example, a researcher may be wondering if certain textual features partition a collection of texts by author or by genre. Pairwise distances alone do not produce any kind of classification. To put a set of distance measurements to work in classification requires additional assumptions, such as a definition of a group or cluster.

The ideas underlying the transition from distances to clusters are, for the most part, common sense. Any clustering of texts should result in texts that are closer to each other (in the distance matrix) residing in the same cluster. There are many ways of satisfying this requirement; there no unique clustering based on distances that is the "best". For our example, we will use the average Euclidean distance between vectors to produce hierarchical groups of clusters. The algorithm for hierarchical clustering of texts is as follows:

1. Start with each document in its own cluster.
2. Until only a single cluster remains,
   * Find the closest clusters and merge them. The distance between two clusters is the average of the nearest and furthest points in the vector.
   * Return a tree containing a record of cluster-merges.

Hierarchical clusterings can be visualised with a tree diagram called a dendrogram. In the code below, we use `scikit-learn's` hierarchical clustering functions to do the clustering.

In [8]:
from scipy.cluster.hierarchy import average, dendrogram

linkage_matrix = average(dist)
dendrogram(linkage_matrix, orientation="right", labels=labels)

{'color_list': ['g', 'r', 'r', 'b', 'b'],
 'dcoord': [[0.0, 3671.1208198335557, 3671.1208198335557, 0.0],
  [0.0, 2774.3294598100611, 2774.3294598100611, 0.0],
  [0.0, 7025.0805232011444, 7025.0805232011444, 2774.3294598100611],
  [0.0, 8631.0583281124582, 8631.0583281124582, 7025.0805232011444],
  [3671.1208198335557,
   11560.953124438209,
   11560.953124438209,
   8631.0583281124582]],
 'icoord': [[5.0, 5.0, 15.0, 15.0],
  [45.0, 45.0, 55.0, 55.0],
  [35.0, 35.0, 50.0, 50.0],
  [25.0, 25.0, 42.5, 42.5],
  [10.0, 10.0, 33.75, 33.75]],
 'ivl': ['CBronte_Jane',
  'CBronte_Villette',
  'CBronte_Professor',
  'Austen_Emma',
  'Austen_Pride',
  'Austen_Sense'],
 'leaves': [3, 5, 4, 0, 1, 2]}

You should see a dictionary containing the numbers used to create the dendrogram. Let's go ahead and plot it. Make sure that you have closed the window for any previous graphs you have created.

In [9]:
plt.tight_layout()  # fixes margins
plt.show()

Close the graph window, and let's try that one more time. You might find this orientation easier to interpret.

In [10]:
dendrogram(linkage_matrix, orientation="top", labels=labels)
plt.tight_layout()  # fixes margins
plt.show()

You can learn how to interpret dendrograms by watching [How to Read a Dendrogram](https://www.youtube.com/watch?v=MX6AUX1b1w0). Alternatively, running the code below will embed the video in this iPython notebook.

In [14]:
from IPython.display import YouTubeVideo
YouTubeVideo("MX6AUX1b1w0")

The iPython `show()` command opens matplotlib in a nice viewer that allows use to save the image and do some zooming. The Python ETE package also has a viewer with some useful functions for studying dendrograms, in particular, the ability to separate overlapping labels. In order to use it, you must save the tree created by scipy to an ETE tree structure and then load it into ETE (which you must first install from the command line using `pip install ete2`).

In [10]:
#ete2 section
root = Tree()
root.dist = 0
root.name = "root"
item2node = {T: root}

to_visit = [T]
while to_visit:
    node = to_visit.pop()
    cl_dist = node.dist /2.0
    for ch_node in [node.left, node.right]:
        if ch_node:
            ch = Tree()
            ch.dist = cl_dist
            ch.name = str(ch_node.id)
            item2node[node].add_child(ch)
            item2node[ch_node] = ch
            to_visit.append(ch_node)

# This is your ETE tree structure
tree = root

# Replace the node labels
for leaf in tree:
    k = leaf.name
    k = int(k)
    leaf.name = labels[k]

from scipy.cluster.hierarchy import linkage, to_tree
from ete2 import Tree, TreeStyle, NodeStyle

T = to_tree(linkage_matrix)

ts = TreeStyle()
ts.show_leaf_name = True
ts.show_branch_length = True
ts.show_scale = False
ts.scale =  None

# Draws nodes as small red spheres of diameter equal to 10 pixels
nstyle = NodeStyle()
nstyle["size"] = 0


# Apply node styles to nodes
for n in tree.traverse():
   n.set_style(nstyle)

# Show the tree in the viewer
tree.show()

# Uncomment and adjust the filename to save the image
#tree.render("mytree.png", w=600, units="px")

You can also convert an ETE tree structure to Newick format for display in other tools. The code below will convert your tree to Newick format and print the Newick string. You can copy and paste it into tools like [EvolView](http://www.evolgenius.info/evolview/#mytrees//). The ETE viewer will also output the tree as a Newick string.

In [ ]:
# Convert the ETE tree to Newick
newick = tree.write()
print(newick)